# 01. Gemini Model
## Gemini 환경 설정
### uv 설치
- vs code 또는 cursor의 agent 기능을 사용하여 처리하면 편하게 설치 가능
- path 설정 필요


### uv를 이용한 가상 환경 생성 및 활성화
- 이전 자료 참고 (https://docs.google.com/presentation/d/1IP_Jr4ZJCanAS3mffSmA0w0dc_N7e29i6x39rDyx9yo/edit?usp=sharing)
```powershell
    uv venv .venv
    .venv\Scripts\activate
```
- editor의 agent를 활용하면 간편히 진행 가능

### Gemini API Key 발급
- `Google AI Studio`에서 API Key 생성 및 발급

<img src="images/google_ai_studio.png" alt="Google AI Studio">

- API Key는 .env에 저장
- .env 파일 생성 후 `GOOGLE_API_KEY`에 입력

<img src="images/google_gemini_api_key_to_env.png" alt="Gemini API Key to .env">


- .env 파일은 git을 통해 공유되지 않도록 반드시 `.gitignore`에 추가 할 것

<img src="images/ignore_env.png" alt="gitignore for .env">


### langchain-google-genai 패키지 설치

- editor의 agent를 사용하여 패키지 설치 진행.

```powershell
    uv add langchain-google-genai python-dotenv
    $env:PATH += ";$env:USERPROFILE\.local\bin"
```

### Select Kernel을 통해 python kernel 지정

- python kernel을 선택

<img src="images/select_kernel.png" alt="Select Kernel">

- 가상 환경을 선택

<img src="images/select_kernel2.png" alt="Select Kernel">



### ipykernel 설치

#### ipykernel이란?
- **IPython Kernel**: Jupyter Notebook에서 Python 코드를 실행하는 핵심 구성요소
- VS Code, Jupyter Lab, Jupyter Notebook에서 Python 코드 셀을 실행할 때 필요

#### 왜 설치해야 하나?
1. **가상환경에서 Jupyter 사용**
   - 가상환경(.venv)에 ipykernel이 없으면 코드 셀 실행 불가
   - VS Code에서 "Select Kernel" 시 가상환경이 목록에 나타나지 않음

2. **패키지 격리**
   - 각 프로젝트별로 독립된 Python 환경 유지
   - 프로젝트에 설치한 패키지들을 Jupyter에서 사용 가능

3. **VS Code Jupyter 확장 호환성**
   - VS Code의 Jupyter 확장이 Python 코드를 실행하려면 kernel 필요
   - ipykernel이 Python 인터프리터와 Jupyter 사이의 브릿지 역할

#### 설치 명령어
```powershell
uv add ipykernel
# 또는
uv pip install ipykernel
```

#### 설치 후 확인
- VS Code에서 "Select Kernel" → 가상환경(.venv) 선택 가능
- Python 코드 셀이 정상적으로 실행됨

## Gemini 사용 간단 예제

In [ ]:
# ---------------------------------------------------------------------
# Gemini API 사용 기본 예제 `hello_gemini` (Python, LangChain)
# - Google Gemini API를 LangChain을 통해 호출
# - 할당량 초과 시 자동 재시도 기능 포함 
# ---------------------------------------------------------------------
import time
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

# 올바른 모델명 사용 (gemini-1.5-pro 또는 gemini-1.5-flash)
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash")  # flash는 더 빠르고 저렴

def call_gemini_with_retry(prompt, max_retries=3, base_delay=60):
    """할당량 초과 시 자동 재시도하는 함수"""
    for attempt in range(max_retries):
        try:
            response = chat.invoke(prompt)
            return response
        except Exception as e:
            error_str = str(e)
            if "429" in error_str and "quota" in error_str.lower():
                if attempt < max_retries - 1:
                    wait_time = base_delay * (2 ** attempt)  # 지수적 백오프
                    print(f"할당량 초과 (시도 {attempt + 1}/{max_retries}). {wait_time}초 후 재시도...")
                    time.sleep(wait_time)
                else:
                    print("최대 재시도 횟수 초과. 나중에 다시 시도해주세요.")
                    raise e
            else:
                print(f"API 호출 중 오류: {e}")
                raise e

# 간단한 질문으로 테스트 (토큰 수 절약)
try:
    response = call_gemini_with_retry("주식 투자를 이용해서 20 퍼센트의 수익을 낼 수 있는 방법을 알려주세요.")
    print("성공적으로 응답을 받았습니다:")
    print(response.content)
except Exception as e:
    print(f"최종 오류: {e}")
    print("\n해결 방법:")
    print("1. 1시간 후 다시 시도")
    print("2. 더 짧은 질문 사용")
    print("3. gemini-1.5-flash 모델 사용 (더 저렴)")
    print("4. Google Cloud Console에서 결제 정보 등록")

### Gemini API 할당량 초과 오류 (429 Error) 해결 방법

#### 오류 원인
- **429 ResourceExhausted**: 무료 할당량 초과
- 1분당 요청 수, 1일 요청 수, 입력 토큰 수 제한 초과

#### 해결 방법

1. **즉시 해결**
   - 잠시 기다렸다가 재시도 (에러 메시지의 retry_delay 시간만큼)
   - 더 짧은 프롬프트 사용
   - `gemini-1.5-flash` 모델 사용 (더 저렴하고 빠름)

2. **장기적 해결**
   - Google Cloud Console에서 결제 정보 등록
   - 유료 플랜으로 업그레이드
   - API 사용량 모니터링

3. **개발 시 권장사항**
   - 테스트용으로는 간단한 질문 사용
   - 요청 간격을 충분히 두기
   - 재시도 로직 구현

### 💡Gemini CLI 활용법 검토
- Gemini CLI의 경우 무료 제공 토큰량이 많고 100만 컨텍스트 윈도우를 지원함
- Gemini 2.5 Pro / Flash 모델 무료로 사용 가능 (일일 1,000회 / 분당 60회 요청 가능)
- 최대 100만 토큰 컨텍스트
- Python의 subprocess로 연동 시도 했지만 제대로 안됨. 
- Google AI API 사용을 권장